In [ ]:
!pip install -q streamlit google-generativeai pillow langchain langgraph

In [ ]:
import os
from getpass import getpass

api = getpass("Enter Google API Key")

if api.strip():
    os.environ["GOOGLE_API_KEY"] = api
    print("✅ API Key set.")


Enter Google API Key··········
✅ API Key set.


In [ ]:
%%writefile agents.py
import os
import google.generativeai as genai
from datetime import datetime

USE_GEMINI = bool(os.environ.get("GOOGLE_API_KEY"))

if USE_GEMINI:
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    model = genai.GenerativeModel("gemini-2.0-flash")
else:
    model = None


#  Agent B: Document Analyst -------------------------------------------------------
from PIL import Image
import io
import google.generativeai as genai

def agent_document_analyst(image_bytes):
    """Extract structured data from image."""

    # Convert bytes → PIL Image (required for Gemini)
    pil_image = Image.open(io.BytesIO(image_bytes))

    if USE_GEMINI:
        response = model.generate_content(
            [
                "Extract: document type, issue date, expiry date, validity and red flags. Respond in pure text.",
                pil_image
            ]
        )
        return response.text


#  Agent C: Compliance Officer -----------------------------------------------------
def agent_compliance_check(extracted_text):
    """Checks against eligibility rules."""

    if USE_GEMINI:
        response = model.generate_content([
            "You are a compliance checker. Given extracted text:",
            extracted_text,
            "Check scheme rule: Income certificate must be issued in last 12 months. "
            "Respond strictly in JSON with fields: decision, code, reason"
        ])
        return response.text


#  Agent D: Communication Agent ----------------------------------------------------
def agent_citizen_message(json_result):
    """Converts decision into citizen-friendly message."""

    if USE_GEMINI:
        response = model.generate_content([
            "Convert the following decision JSON into a SHORT, SIMPLE and POLITE message in Hindi:",
            json_result
        ])
        return response.text


Writing agents.py


In [ ]:
%%writefile app.py
import streamlit as st
from agents import (
    agent_document_analyst,
    agent_compliance_check,
    agent_citizen_message
)

st.set_page_config(page_title="GovVerify AI", layout="wide")
st.title("🏛️ GovVerify AI – Automated Document Verification")

uploaded = st.file_uploader("Upload a document (image/PDF)", type=["png","jpg","jpeg","pdf"])

if uploaded:
    st.image(uploaded.read(), caption="Uploaded Document", use_container_width=True)

    with st.spinner("Analyzing document..."):
        analyst_output = agent_document_analyst(uploaded.getvalue())

    st.subheader("🔍 Document Analyst Output")
    st.code(analyst_output)

    with st.spinner("Checking compliance..."):
        compliance = agent_compliance_check(analyst_output)

    st.subheader("⚖ Compliance Result")
    st.code(compliance)

    with st.spinner("Preparing citizen message..."):
        citizen_msg = agent_citizen_message(compliance)

    st.subheader("📢 Message for Applicant")
    st.success(citizen_msg)


Writing app.py


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb


Selecting previously unselected package cloudflared.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!streamlit run app.py --server.port 8501 > /content/streamlit.log 2>&1 &

import time
time.sleep(3)

print("✓ Streamlit started")


✓ Streamlit started


In [ ]:
!cloudflared tunnel \
    --url http://localhost:8501 \
    --protocol http2 \
    --edge-ip-version auto \
    --no-autoupdate \
    > /content/tunnel.log 2>&1 &

print("⏳ Starting Cloudflare Tunnel...")
time.sleep(8)


⏳ Starting Cloudflare Tunnel...


In [ ]:
import time, re

with open("/content/tunnel.log") as f:
    log = f.read()

urls = re.findall(r"https://[-0-9a-z]+\.trycloudflare\.com", log)

if urls:
    print("🌍 Your public Streamlit App URL:")
    print(urls[-1])
else:
    print("⚠ No URL found yet. Rerun this cell in 5 seconds.")


🌍 Your public Streamlit App URL:
https://mrna-etc-seafood-female.trycloudflare.com
